<a href="https://colab.research.google.com/github/RyanAleixo/Profiss-o-Cientista-de-Dados-v2/blob/main/MOD_35_EXERCICIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MÓDULO 35 - Cross Validation**

Nesta tarefa, você trabalhará com uma base de dados que contém informações sobre variáveis ambientais coletadas para a detecção de incêndios. O objetivo é utilizar técnicas de validação cruzada (cross-validation) para avaliar a performance de um modelo de classificação na previsão da ocorrência de um incêndio com base nas variáveis fornecidas.


Descrição da Base de Dados
A base de dados contém as seguintes variáveis:

Unnamed:0: Índice (não é uma variável útil para o modelo)

UTC: Tempo em Segundos UTC

Temperature[C]: Temperatura do Ar (em graus Celsius)

Humidity[%]: Umidade do Ar (em porcentagem)

TVOC[ppb]: Total de Compostos Orgânicos Voláteis (medido em partes por bilhão)

eCO2[ppm]: Concentração equivalente de CO2 (medido em partes por milhão)

Raw H2: Hidrogênio molecular bruto, não compensado

Raw Ethanol: Etanol gasoso bruto

Pressure[hPA]: Pressão do Ar (em hectopascais)

PM1.0: Material particulado de tamanho < 1,0 µm

PM2.5: Material particulado de tamanho >1,0 µm e < 2,5 µm

NC0.5: Concentração numérica de material particulado de tamanho < 0,5 µm

NC1.0: Concentração numérica de material particulado de tamanho 0,5 µm < 1,0 µm

NC2.5: Concentração numérica de material particulado de tamanho 1,0 µm < 2,5 µm

CNT: Contador de amostras


E a variável alvo:

Fire Alarm: Indicador binário de incêndio (1 se houver incêndio, 0 caso contrário)

O objetivo desta tarefa é aplicar a técnica de validação cruzada (cross-validation) para avaliar a performance de um modelo de classificação. A validação cruzada ajudará a garantir que o modelo seja avaliado de maneira robusta e generalize bem para dados não vistos.

In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from google.colab import files
import io


# 1 - Carregue a base de dados, verifique os tipos de dados e também se há presença de dados faltantes ou nulos.

In [2]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
df = pd.read_csv(io.BytesIO(uploaded[file_name]), delimiter=',')
print(df.head())

Saving Cientista de dados M35 - smoke_detection_iot.csv to Cientista de dados M35 - smoke_detection_iot (1).csv
   Unnamed: 0         UTC  Temperature[C]  Humidity[%]  TVOC[ppb]  eCO2[ppm]  \
0           0  1654733331          20.000        57.36          0        400   
1           1  1654733332          20.015        56.67          0        400   
2           2  1654733333          20.029        55.96          0        400   
3           3  1654733334          20.044        55.28          0        400   
4           4  1654733335          20.059        54.69          0        400   

   Raw H2  Raw Ethanol  Pressure[hPa]  PM1.0  PM2.5  NC0.5  NC1.0  NC2.5  CNT  \
0   12306        18520        939.735    0.0    0.0    0.0    0.0    0.0    0   
1   12345        18651        939.744    0.0    0.0    0.0    0.0    0.0    1   
2   12374        18764        939.738    0.0    0.0    0.0    0.0    0.0    2   
3   12390        18849        939.736    0.0    0.0    0.0    0.0    0.0    3   
4 

In [3]:
print("--- Tipos de Dados ---")
print(df.dtypes)

print("\n--- Dados Faltantes (Soma por Coluna) ---")
print(df.isnull().sum())


print("\n--- Primeiras 5 linhas da base ---")
print(df.head())


print("\n--- Resumo Estatístico ---")
print(df.describe())

--- Tipos de Dados ---
Unnamed: 0          int64
UTC                 int64
Temperature[C]    float64
Humidity[%]       float64
TVOC[ppb]           int64
eCO2[ppm]           int64
Raw H2              int64
Raw Ethanol         int64
Pressure[hPa]     float64
PM1.0             float64
PM2.5             float64
NC0.5             float64
NC1.0             float64
NC2.5             float64
CNT                 int64
Fire Alarm          int64
dtype: object

--- Dados Faltantes (Soma por Coluna) ---
Unnamed: 0        0
UTC               0
Temperature[C]    0
Humidity[%]       0
TVOC[ppb]         0
eCO2[ppm]         0
Raw H2            0
Raw Ethanol       0
Pressure[hPa]     0
PM1.0             0
PM2.5             0
NC0.5             0
NC1.0             0
NC2.5             0
CNT               0
Fire Alarm        0
dtype: int64

--- Primeiras 5 linhas da base ---
   Unnamed: 0         UTC  Temperature[C]  Humidity[%]  TVOC[ppb]  eCO2[ppm]  \
0           0  1654733331          20.000        57.36 

Para a coluna Fire Alarm, por conta do espaçamento talvez seja util renomear o nome da coluna utilizando:

df.rename(columns={'Fire Alarm': 'Fire_Alarm'}, inplace=True)

In [4]:
df.rename(columns={'Fire Alarm': 'Fire_Alarm'}, inplace=True)

df.drop(columns=['Unnamed: 0', 'UTC', 'CNT'], inplace=True)

print("Colunas atuais na base:")
print(df.columns.tolist())

print("\nDistribuição da variável Fire_Alarm:")
print(df['Fire_Alarm'].value_counts(normalize=True))

Colunas atuais na base:
['Temperature[C]', 'Humidity[%]', 'TVOC[ppb]', 'eCO2[ppm]', 'Raw H2', 'Raw Ethanol', 'Pressure[hPa]', 'PM1.0', 'PM2.5', 'NC0.5', 'NC1.0', 'NC2.5', 'Fire_Alarm']

Distribuição da variável Fire_Alarm:
Fire_Alarm
1    0.714626
0    0.285374
Name: proportion, dtype: float64


# 2 - Para essa base, onde você realizará as previsões de fire alarm, qual modelo de machine learning você aplicará? Justifique.

Para esta base de detecção de incêndio, o modelo ideal é o Random Forest Classifier. Justificativa:

Lidando com Dados Complexos: Sensores de gás e temperatura possuem relações não lineares; o Random Forest captura essas nuances sem precisar de ajustes manuais.

Resistência a Ruídos: Equipamentos IoT podem gerar leituras "sujas" (outliers); a estrutura de múltiplas árvores do modelo filtra esses erros, tornando a detecção mais estável.

Foco em Generalização: Como o tema é Cross-Validation, o Random Forest é perfeito, pois ele naturalmente evita o sobreajuste (overfitting) ao combinar várias árvores de decisão.

Importância das Variáveis: O modelo permite identificar qual sensor (como TVOC ou Umidade) é o mais decisivo para o alarme, facilitando a interpretação do problema.


# 3 - Separe a base em Y e X e já rode a instância do modelo que você utilizará.

In [6]:
X = df.drop(columns=['Fire_Alarm'])
y = df['Fire_Alarm']

modelo_incendio = RandomForestClassifier(n_estimators=100, random_state=42)

print("Separação concluída!")
print(f"Variáveis preditoras (X): {X.columns.tolist()}")
print(f"Alvo (y): Fire_Alarm")

Separação concluída!
Variáveis preditoras (X): ['Temperature[C]', 'Humidity[%]', 'TVOC[ppb]', 'eCO2[ppm]', 'Raw H2', 'Raw Ethanol', 'Pressure[hPa]', 'PM1.0', 'PM2.5', 'NC0.5', 'NC1.0', 'NC2.5']
Alvo (y): Fire_Alarm


# 4 - Defina o número de Folds e rode o modelo com a validação cruzada.

In [7]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_score(modelo_incendio, X, y, cv=kf, scoring='accuracy')

print("Acurácia em cada Fold:")
for i, score in enumerate(scores, 1):
    print(f"Dobras {i}: {score:.4f}")

Acurácia em cada Fold:
Dobras 1: 1.0000
Dobras 2: 0.9998
Dobras 3: 1.0000
Dobras 4: 1.0000
Dobras 5: 0.9998


# 5 - Avalie a pontuação de cada modelo e ao final a validação final da média.

In [8]:
media_acuracia = scores.mean()
desvio_padrao = scores.std()

# 2. Exibindo os resultados finais
print(f"--- Relatório de Performance ---")
print(f"Média da Acurácia: {media_acuracia:.4f} ({media_acuracia*100:.2f}%)")
print(f"Desvio Padrão: {desvio_padrao:.4f}")

# 3. Validação da robustez
if desvio_padrao < 0.05:
    print("\nConclusão: O modelo é estável e apresenta baixa variância entre os testes.")
else:
    print("\nConclusão: O modelo apresenta alta variância, sugerindo necessidade de ajuste nos dados.")

--- Relatório de Performance ---
Média da Acurácia: 0.9999 (99.99%)
Desvio Padrão: 0.0001

Conclusão: O modelo é estável e apresenta baixa variância entre os testes.
